In [ ]:
import matplotlib.pyplot as plt
import os
import sys

sys.path.append('..')
os.environ.update(dict(CUDA_VISIBLE_DEVICES='3'))

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm import tqdm

from torch.distributions import MultivariateNormal
from models.Res import ResNet, resnet50

from analysis import *

In [ ]:
class ResNetWrapper(nn.Module):
    def __init__(self, net: ResNet, i_block=0):
        super().__init__()
        self.net = net
        self.i_block = i_block
    
    def forward(self, x, return_feature=False, return_feature_only=False):
        if self.i_block <= 0:
            x = self.net.conv1(x)
            x = self.net.bn1(x)
            x = self.net.relu(x)
            x = self.net.maxpool(x)

        if self.i_block <= 1:
            x = self.net.layer1(x)
            
        if self.i_block <= 2:
            x = self.net.layer2(x)

        if self.i_block <= 3:
            x = self.net.layer3(x)
        # if return_feature:
        #     feature = x
        if self.i_block <= 4:
            x = self.net.layer4(x)
            x = self.net.avgpool(x)
            x = x.reshape(x.size(0), -1)
            
        if return_feature:
            feature = x
        x = self.net.fc(x)

        if return_feature:
            if return_feature_only:
                return feature
            else:
                return x, feature

        else:
            return x